In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
This notebook demonstrates how to retrieve Sentinel2 base imagery for pixel-based crop type predictions.
"""

import sys
import os

# Set up imports
project_root = os.path.abspath("..") 
sys.path.append(project_root)  

In [ ]:
from preprocessing.datasources import (
    query_stac_api, 
    list_folders_second_to_deepest_level, 
    get_existing_data, 
    get_directory_size,
    unique_indices, process_result
    )

#### Split data to be processed into chunks of bboxes & years

In [ ]:
# Split data to be processed into chunks of bboxes & years

# Specify the root path
CDL_path = "../data/CDL_unique_scene.parquet/"
#CDL_path = "../data/CDL_samples.parquet/"   

# Define the target depth
target_depth = 2  # Second-to-deepest level

# Get the list of folders at the target depth recursively
folder_paths = list_folders_second_to_deepest_level(CDL_path, [], 0, target_depth)

# Convert folder paths to partition value dictionaries
df_train_partition_values_list = []
for path in folder_paths:
    segments = path.split(os.sep)
    partition_values = {}
    for segment in segments:
        if '=' in segment:
            key, value = segment.split('=')
            partition_values[key] = value
    df_train_partition_values_list.append(partition_values)


In [ ]:
folder_paths

In [ ]:
df_train_partition_values_list

#### Retrive existing data (to avoid reprocessing)

In [ ]:
# This function is only needed/used for restarting processing after stopping for some reason (start where code left off)
# e.g. full file
s2_file_path = '../data/dbfs_s2_unique_scene.parquet/'

print(get_existing_data(s2_file_path))

In [ ]:
try:
    dir_size_gb = get_directory_size(s2_file_path) / (1024 ** 3)
    print(f"Size of '{s2_file_path}' is {dir_size_gb:.2f} GB")
except FileNotFoundError:
    print("Directory does not exist.")


#### Engine/Loop to retrieve and sample Sentinel-2 data

In [ ]:
import multiprocessing

assets_list = ['scl', 'coastal', 'blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir', 'nir08', 'nir09', 'swir16', 'swir22']
scl_exclude_list = [0, 1, 7, 8, 9, 11] # ignore certain scl layer values....
# SCL_color_mappings = {
#   0: # No Data (Missing data) - black  
#   1: # Saturated or defective pixel - red 
#   2: # Topographic casted shadows ("Dark features/Shadows" for data before 2022-01-25) - very dark grey
#   3: # Cloud shadows - dark brown
#   4: # Vegetation - green
#   5: # Not-vegetated - dark yellow
#   6: # Water (dark and bright) - blue
#   7: # Unclassified - dark grey
#   8: # Cloud medium probability - grey
#   9: # Cloud high probability - white
#   10: # Thin cirrus - very bright blue
#   11: # Snow or ice - very bright pink
# }

In [ ]:
import time
from datetime import timedelta

existing_s2_dates = get_existing_data(s2_file_path)
s2_file_path = '../data/s2_unique_scene.parquet/' # output path

lock = multiprocessing.Lock()

start_time = time.time()
successful_scenes = 0
failed_scenes = 0

for el in df_train_partition_values_list[:1]:  
    bbox = el['bbox']
    year = el['year']
    CDL_parts_path = os.path.join(CDL_path, f"bbox={bbox}", f"year={year}")

    try:
        print(f"[INFO] Trying delimiter ', ': {bbox}")
        bbox_tuple = tuple([int(x) for x in bbox.split(', ')])
        print("[OK] Parsed bbox using delimiter ', '")
    except ValueError:
        print(f"[INFO] Trying delimiter ',': {bbox}")
        bbox_tuple = tuple([int(x) for x in bbox.split(',')])
        print("[OK] Parsed bbox using delimiter ','")

    results = query_stac_api(
        bounds=bbox_tuple,
        epsg4326=False,
        start_date=f"{year}-01-01T00:00:00Z",
        end_date=f"{year}-12-31T23:59:59Z"
    )

    results = unique_indices(results)
    print(f"🎯 Total scenes to process: {len(results)}")

    def process_results_in_parallel(result):
        status = process_result(result, existing_s2_dates, CDL_parts_path,
                                assets_list, scl_exclude_list,
                                s2_file_path, bbox, year, lock)
        return status

    use_mp = True
    if use_mp:
        with multiprocessing.Pool(processes=multiprocessing.cpu_count(),
                                  maxtasksperchild=1) as pool:
            statuses = pool.map(process_results_in_parallel, results)
    else:
        from concurrent.futures import ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=4) as executor:
            statuses = list(executor.map(process_results_in_parallel, results))

    successful_scenes += sum(1 for s in statuses if s == 0)
    failed_scenes += sum(1 for s in statuses if s != 0)

    del results

total_time = timedelta(seconds=int(time.time() - start_time))

print("\n✅ Done.")
print(f"⏱️ Total time: {total_time}")
print(f"📸 Scenes processed: {successful_scenes}")
print(f"❌ Scenes failed: {failed_scenes}")

#### Retrieve a tile per scene date example

In [ ]:
import requests

url = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/S/YV/2019/6/S2B_15SYV_20190625_1_L2A/SCL.tif"
output_path = "../data/S2B_15SYV_20190625_1_L2A_SCL.tif"

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(output_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("✅ SCL.tif downloaded.")


In [ ]:
import rasterio
import matplotlib.pyplot as plt

scl_path = "../data/S2B_15SYV_20190625_1_L2A_SCL.tif"

with rasterio.open(scl_path) as src:
    scl_array = src.read(1)
    plt.figure(figsize=(10, 8))
    plt.imshow(scl_array, cmap='tab20', vmin=0, vmax=11)
    plt.colorbar(label='SCL class')
    plt.title("Scene Classification Layer (SCL) - 2019-06-25")
    plt.axis('off')
    plt.show()


In [ ]:
import numpy as np
unique, counts = np.unique(scl_array, return_counts=True)
scl_stats = dict(zip(unique, counts))
print("📊 SCL value counts:", scl_stats)